In [2]:
import os
import json

import numpy as np
import pandas as pd
import pennylane as qml

In [43]:
folder_path = r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\SUSY QM\PennyLane\ADAPT-VQE\Files\AHO"
file_paths = os.listdir(folder_path)

In [44]:
json_files = [f for f in file_paths if f.endswith('.json')]
json_files

['AHO_16.json', 'AHO_2.json', 'AHO_32.json', 'AHO_4.json', 'AHO_8.json']

In [ ]:
for file in json_files:

    data_path = os.path.join(folder_path,file)
    data_dict = {}

    with open(data_path, 'r') as file:
            data = json.load(file)

    cutoff = data['cutoff']
    potential = data['potential']

    print(f"Finding optimal circuit for {potential} - cutoff {cutoff}")

    num_qubits = int(np.log2(cutoff)) + 1
    min_eigenvalue = min(data['exact_eigenvalues'])

    # Only consider data that has converge to min energy
    mask = []
    for d in data['min_energies']:
        if abs(d - min_eigenvalue) < 1e-6:
            mask.append(True)
        else:
            mask.append(False)

    min_ops = [op_list for op_list, keep in zip(data['op_list'], mask) if keep]
    min_len = min([len(ops) for ops in min_ops])
    reduced = [ops for ops in min_ops if len(ops) == min_len]

    # Take the unique circuits
    unique_set = set()
    unique_op_lists = []

    for ops in reduced:
        
        set_key = tuple((op['name'], tuple(op['wires'])) for op in ops)
        
        if set_key not in unique_set:
            unique_set.add(set_key)
            unique_op_lists.append(ops)
    
    
    basis_state = data['basis_state']

    data_dict['potential'] = potential
    data_dict['cutoff'] = cutoff
    data_dict['basis_state'] = basis_state
    data_dict['circuits'] = unique_op_lists

    file_path = os.path.join(folder_path, f"data_{cutoff}.txt")

    with open(file_path, "w") as file:
        json.dump(data_dict, file, indent=4)

    # Define the device
    dev = qml.device("default.qubit", wires=num_qubits)
    
    for i in range(len(unique_op_lists)):
    
        @qml.qnode(dev)
        def circuit():

            qml.BasisState(basis_state, wires=range(num_qubits))

            for op_dict in unique_op_lists[i]:
                op = getattr(qml, op_dict["name"])
                op(op_dict['param'], wires=op_dict['wires'])

            return qml.state()
        
        circuit_diagram = qml.draw(circuit)()
        mpl_diagram, ax = qml.draw_mpl(circuit, style='pennylane')()

        diagram_path = os.path.join(folder_path, "circuitDiagrams")
        os.makedirs(diagram_path, exist_ok=True)
        mpl_diagram.savefig(diagram_path + f"\\{potential}_{cutoff}_{i}.png")


        with open(file_path, "a", encoding="utf-8") as file:
            file.write("\n###############################\n")
            file.write(circuit_diagram)
            file.write("\n###############################\n")


In [50]:
potentials = ['QHO','AHO', 'DW']
cutoffs = [2,4,8,16,32]

d_potentials = []
d_cutoffs = []
d_basis = []
d_ops_list = []
d_exact_e = []
d_energies = []


for potential in potentials:
    for cutoff in cutoffs:
        data_path = os.path.join(r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\SUSY QM\PennyLane\ADAPT-VQE\Files", potential, f"{potential}_{cutoff}.json")

        with open(data_path, 'r') as file:
            data = json.load(file)

        min_eigenvalue = min(data['exact_eigenvalues'])


        d_potentials.append(potential)
        d_cutoffs.append(cutoff)
        d_basis.append(data['basis_state'])
        d_exact_e.append(min_eigenvalue)

        # Only consider data that has converge to min energy
        mask = []
        for d in data['min_energies']:
            if abs(d - min_eigenvalue) < 1e-6:
                mask.append(True)
            else:
                mask.append(False)

        # Clean and prepare circuit data
        min_ops = [op_list for op_list, keep in zip(data['op_list'], mask) if keep]
        min_len = min([len(ops) for ops in min_ops])
        reduced = [ops for ops in min_ops if len(ops) == min_len]
        energies = [min(op_list) for op_list, keep in zip(data['all_energies'], mask) if keep]

        d_energies.append(min(energies))


        # Take the unique circuits
        unique_set = set()
        unique_op_lists = []

        for ops in reduced:
            
            set_key = tuple((op['name'], tuple(op['wires'])) for op in ops)
            
            if set_key not in unique_set:
                unique_set.add(set_key)
                unique_op_lists.append(ops)

        all_labels = []
        for i in range(len(unique_op_lists)):
            op_labels = []

            for op_dict in unique_op_lists[i]:
            
                wires = op_dict['wires']
                if len(wires) > 1:
                    label = f'{op_dict['name']}({str(wires[0])},{str(wires[1])})'
                else:
                    label = f"{op_dict['name']}({str(wires[0])})"

                op_labels.append(label)

            all_labels.append(op_labels)

        d_ops_list.append(all_labels)
    

In [51]:
data_dict = {'Potential':d_potentials,
      'Cutoff': d_cutoffs,
      'Basis State': d_basis,
      'Operators': d_ops_list,
      'Min E Exact': d_exact_e,
      'Min E': d_energies}

In [52]:
df = pd.DataFrame(data_dict)
df

,Potential,Cutoff,Basis State,Operators,Min E Exact,Min E
0,QHO,2,"[1, 0]",[[RY(0)]],0.000000,0.000000
1,QHO,4,"[1, 0, 0]",[[RY(0)]],0.000000,0.000000
2,QHO,8,"[1, 0, 0, 0]",[[RY(0)]],0.000000,0.000000
3,QHO,16,"[1, 0, 0, 0, 0]",[[RY(0)]],0.000000,0.000000
4,QHO,32,"[1, 0, 0, 0, 0, 0]",[[RY(0)]],0.000000,0.000000
5,AHO,2,"[1, 0]",[[RY(0)]],-0.437500,-0.437500
6,AHO,4,"[1, 0, 0]",[[RY(1)]],-0.164785,-0.164785
7,AHO,8,"[1, 0, 0, 0]","[[RY(1), RY(2)]]",0.032010,0.032010
8,AHO,16,"[1, 0, 0, 0, 0]","[[RY(2), RY(3)]]",-0.001167,-0.001167
9,AHO,32,"[1, 0, 0, 0, 0, 0]","[[RY(3), RY(4)]]",0.000006,0.000006


In [31]:
from tabulate import tabulate

In [35]:
latex_table = tabulate(df.values, headers=df.columns, tablefmt="latex")#, floatfmt=".3f")
print(latex_table)


\begin{tabular}{lrllrr}
\hline
 Potential   &   Cutoff & Basis State        & Operators                                                                                                                                                            &   Min E Exact &        Min E \\
\hline
 QHO         &        2 & [1, 0]             & [['RY(0)']]                                                                                                                                                          &   0           &  0           \\
 QHO         &        4 & [1, 0, 0]          & [['RY(0)']]                                                                                                                                                          &   0           &  0           \\
 QHO         &        8 & [1, 0, 0, 0]       & [['RY(0)']]                                                                                                                                                          &   0       